In [1]:
%load_ext autoreload
%autoreload 2
import rootutils
import os
from pathlib import Path

root = rootutils.setup_root(
    os.path.abspath(""), indicator=".project-root", pythonpath=True
)

import time

import src.data.components.processed_tile_group_pb2 as pbf
import pandas as pd
import plotly.express as px
import umap.umap_ as umap
from src.modules.tag_encoder import TagEncoder
from src.data.tiles_datamodule import TilesDataModule

In [2]:
tile_group = pbf.TileGroup()
working_dir = os.getcwd()
print(os.getcwd())
with open(root / Path("data/tiles/sthlm/processed/14_9006_11577.pbf"), "rb") as f:
    tile_group.ParseFromString(f.read())

/Users/theo/Documents/courses/current/master-thesis/geojepa/notebooks


In [3]:
start = time.time()
encoder = TagEncoder(embedding_file=root / Path("data/tiles/embeddings.pkl"))
print(f"Time to load encoder: {time.time() - start} s")

Time to load encoder: 0.5498270988464355 s


In [26]:

%autoreload 2
from src.modules.tag_encoder import TagEncoder

encoder = TagEncoder(embedding_file=root / Path("data/tiles/embeddings.pkl"))
dataset = TilesDataModule(root / Path("data/tiles/sthlm/processed"), num_workers=0)
dataset.setup()
loader = dataset.test_dataloader()
tag_embeddings = []
tag_strings = []
tiles = 0

start_time = time.time()
for i, batch in enumerate(loader):
    tiles += len(batch.tile_coords)
    embs, strings = encoder(batch.tags)
    for i in range(len(strings)):
        for j in range(len(strings[i])):
            # if strings[i][j] != "" and "speed" in strings[i][j]:
            if strings[i][j] != "":
                tag_embeddings.append(embs[i][j].detach().numpy())
                tag_strings.append(strings[i][j])
end_time = time.time()
print("Time: ", (end_time - start_time) * 1e3, "ms")
print("Time: ", (end_time - start_time) * 1e3 / tiles, "ms per tile")
print("Tiles: ", tiles)


def flatten(matrix):
    return [item for row in matrix for item in row]


# for batch in dataset.test_dataloader():
#     print("batch_size", batch.size)
#     print("nbr_features", batch.nbr_features)
#     print("tag_count", len(flatten(batch.tags)))
#     print("geo_count", batch.node_to_feature[-1])
#     print(batch.nodes)
#     print(batch.node_to_feature)

print(len(tag_strings), "features with embedding")
not_found_tags = sorted(list(encoder.not_found_tags.items()), key=lambda x: -x[1])
print(len(encoder.no_embs_found_on_feat), "features without embeddings")
print(len(not_found_tags), "unqiue tags not found")
print(not_found_tags[:5], "\n...\n", not_found_tags[-5:])

Time:  310.27984619140625 ms
Time:  1.2167837105545343 ms per tile
Tiles:  255
11572 features with embedding
123 features without embeddings
77 unqiue tags not found
[(('lst:metodbeskrivning', 'Status på gränsdragning enligt VIC-natur: Justerad mot fastighetskarta'), 19), (('lst:url', 'http://www.ab.lst.se/templates/InformationPage____2017.asp'), 17), (('interval:peak', '10'), 16), (('interval:evening', '30'), 16), (('lst:anmärkning', 'Dossienr. Lst: 0115-02-002'), 11)] 
...
 [(('description:en', 'For bird watching'), 1), (('description:en', 'Probably dates from 1260s. Separate 1661 wooden bell tower from. 4 run stones.'), 1), (('description:en', 'Access point for Angarnssjöängen Naturreservat'), 1), (('description:en', 'parking for disabled persons'), 1), (('accessibility', 'Next to public footpath'), 1)]


In [5]:
# df = pd.DataFrame(np.nan, index=range(len(embeddings_array)), columns=range(3))
df = pd.DataFrame({"matching_string": tag_strings})
# Apply UMAP to reduce to 2D
umap_model = umap.UMAP(n_components=2, random_state=42, n_neighbors=60)
umap_results = umap_model.fit_transform(tag_embeddings)

# Add UMAP results to DataFrame
df["umap-2d-one"] = umap_results[:, 0]
df["umap-2d-two"] = umap_results[:, 1]

# Create an interactive Plotly scatter plot
fig = px.scatter(df, x="umap-2d-one", y="umap-2d-two", hover_name="matching_string")

fig.write_html("plots/" + "features-speed" + ".html")

/opt/homebrew/anaconda3/envs/geojepa-ipynb/lib/python3.10/site-packages/umap/umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [6]:
%autoreload 2
from src.datasets.tiles import TilesDataModule

dataset = TilesDataModule("../../../osm_pipeline/tiles/sthlm/processed", num_workers=0)
dataset.setup()

ModuleNotFoundError: No module named 'src.datasets'

In [ ]:
# df = pd.DataFrame(np.nan, index=range(len(embeddings_array)), columns=range(3))
# Apply UMAP to reduce to 2D
umap_model = umap.UMAP(n_components=2, random_state=42)

loader = dataset.train_dataloader()
tag_embeddings = []
tag_strings = []
for i, batch in enumerate(loader):
    embs, strings = encoder(batch.tags)
    for j, s in enumerate(strings):
        tag_strings.append(s)
        tag_embeddings.append(embs[j].numpy())

umap_results = umap_model.fit_transform(tag_embeddings)

df = pd.DataFrame({"matching_string": tag_strings})
# Add UMAP results to DataFrame
df["umap-2d-one"] = umap_results[:, 0]
df["umap-2d-two"] = umap_results[:, 1]

# Create an interactive Plotly scatter plot
fig = px.scatter(df, x="umap-2d-one", y="umap-2d-two", hover_name="matching_string")

fig.write_html("plots/" + "tiles" + ".html")